In [ ]:
%pip install pandas

In [ ]:
%run ../load_comunas_servicios.py "../../datasets/comunas_por_zona.csv"

comunas_servicios_df.rename(columns={"zona_id": "capitulo", "zona": "nombre_capitulo"}, inplace=True)
print(comunas_servicios_df.info())
comunas_servicios_df.head()

## CASEN 2017


In [ ]:
import pandas as pd

FONASA_A = 1
FONASA_B = 2
FONASA_C = 3
FONASA_D = 4
FONASA_NO_SABE = 5
FONASA_GRUPOS = [FONASA_A, FONASA_B, FONASA_C, FONASA_D, FONASA_NO_SABE]
PREGUNTA_PREVISION = "s12"
SI = 1
NO = 2
NS = 9
NR = 9
FACTOR_EXPANSION_REGIONAL = "expr"
PROBLEMAS_LLEGAR_CONSULTA = "s18a"
PROBLEMAS_CONSEGUIR_CITA = "s18b"
PROBLEMAS_SER_ATENDIDO = "s18c"
PROBLEMAS_PAGAR_POR_COSTO = "s18d"
PROBLEMAS_ENTREGA_MEDICAMENTOS = "s18e"


casen_2017_df = pd.read_csv(
    "../../datasets/casen/casen_2017.csv",
    sep=",",
    dtype={"expr": "int", "expc": "int", "comuna": "int"},
    converters={
        "folio": lambda folio: folio.split(".")[0],
        "id_vivienda": lambda id: id.split(".")[0],
    },
)
casen_2017_df.drop(columns=["zona"], inplace=True)

print(len(casen_2017_df))
casen_2017_df.head()

In [ ]:
casen_2017_df.info()

In [ ]:
print(len(casen_2017_df))

In [ ]:
casen_2017_df["folio"].value_counts()

In [ ]:
casen_2017_df = casen_2017_df[casen_2017_df[PREGUNTA_PREVISION].isin(FONASA_GRUPOS)]

In [ ]:
casen_2017_df["comuna"].value_counts()

In [ ]:
casen_2017_df[PROBLEMAS_LLEGAR_CONSULTA].value_counts()

In [ ]:
casen_2017_df = pd.merge(
    casen_2017_df,
    comunas_servicios_df[["capitulo", "nombre_capitulo", "comuna_id"]],  # type: ignore
    left_on="comuna",
    right_on="comuna_id",
    how="left",
)
casen_2017_df.drop(columns=["comuna_id"], inplace=True)

In [ ]:
casen_2017_df

In [ ]:
casen_2017_df.query("capitulo == 20")["s18a"].value_counts(dropna=False)

In [ ]:
print(len(casen_2017_df))

In [ ]:
def calculate_problem_metrics(s, question, problem):
    total_respuestas = s[s[question].notna()][FACTOR_EXPANSION_REGIONAL].sum()
    total_si = s[s[question] == SI][FACTOR_EXPANSION_REGIONAL].sum()
    total_no = s[s[question] == NO][FACTOR_EXPANSION_REGIONAL].sum()
    total_nsnr = s[s[question].isin([NS, NR])][FACTOR_EXPANSION_REGIONAL].sum()

    return {
        f"{problem}_respuestas": total_respuestas,
        f"{problem}_si": total_si,
        f"{problem}_no": total_no,
        f"{problem}_nsnr": total_nsnr,
        f"{problem}_si_porcentaje": (
            total_si / total_respuestas * 100 if total_respuestas else 0
        ),
        f"{problem}_no_porcentaje": (
            total_no / total_respuestas * 100 if total_respuestas else 0
        ),
        f"{problem}_nsnr_porcentaje": (
            total_nsnr / total_respuestas * 100 if total_respuestas else 0
        ),
    }


def calculate_servicio_metrics(s):
    total_fonasa = s[FACTOR_EXPANSION_REGIONAL].sum()
    problemas_llegar = calculate_problem_metrics(
        s, PROBLEMAS_LLEGAR_CONSULTA, "problemas_llegar_consulta"
    )
    problemas_conseguir_cita = calculate_problem_metrics(
        s, PROBLEMAS_CONSEGUIR_CITA, "problemas_conseguir_cita"
    )
    problemas_ser_atendido = calculate_problem_metrics(
        s, PROBLEMAS_SER_ATENDIDO, "problemas_ser_atendido"
    )
    problemas_pagar_por_costo = calculate_problem_metrics(
        s, PROBLEMAS_PAGAR_POR_COSTO, "problemas_pagar_por_costo"
    )
    problemas_entrega_medicamentos = calculate_problem_metrics(
        s, PROBLEMAS_ENTREGA_MEDICAMENTOS, "problemas_entrega_medicamentos"
    )

    return pd.Series(
        {
            "total_fonasa": total_fonasa,
            **problemas_llegar,
            **problemas_conseguir_cita,
            **problemas_ser_atendido,
            **problemas_pagar_por_costo,
            **problemas_entrega_medicamentos,
        }
    )


casen_2017_grouped_servicio_df = (
    casen_2017_df.groupby(["capitulo", "nombre_capitulo"])
    .apply(calculate_servicio_metrics)
    .reset_index()
)

casen_2017_grouped_servicio_df.info()

In [ ]:
casen_2017_grouped_servicio_df.head()

In [ ]:
print(casen_2017_grouped_servicio_df["total_fonasa"].sum())
print(casen_2017_grouped_servicio_df["problemas_llegar_consulta_si"].sum())

## CASEN 2022


In [ ]:
import pandas as pd

FONASA = 1
PREGUNTA_PREVISION = "s13"

casen_2022_df = pd.read_stata(
    "../../datasets/casen/casen_2022.dta",
    convert_categoricals=False,
)

In [ ]:
provincia_comuna_df = pd.read_stata(
    "../../datasets/casen/provincia_comuna_casen_2022.dta"
)
# provincia_comuna_df["folio"].value_counts()
provincia_comuna_df.head()

In [ ]:
casen_2022_df = pd.merge(
    casen_2022_df, provincia_comuna_df[["folio", "comuna"]], on="folio"
)
casen_2022_df

In [ ]:
casen_2022_df["folio"].value_counts()

In [ ]:
print(len(casen_2022_df))
casen_2022_df.head()

In [ ]:
casen_2022_df["region"].unique()

In [ ]:
casen_2022_df["comuna"]

In [ ]:
print(PREGUNTA_PREVISION)
casen_2022_df = casen_2022_df[casen_2022_df[PREGUNTA_PREVISION] == FONASA]

In [ ]:
casen_2022_df["comuna"].value_counts()

In [ ]:
comunas_servicios_df

In [ ]:
casen_2022_df = pd.merge(
    casen_2022_df,
    comunas_servicios_df,  # type: ignore
    on="comuna",
)

casen_2022_df.drop(columns=["comuna_id"], inplace=True)

In [ ]:
casen_2022_df

In [ ]:
casen_2022_df[FACTOR_EXPANSION_REGIONAL].sum()

In [ ]:
NS = -88
NR = -99
PROBLEMAS_LLEGAR_CONSULTA = "s19a"
PROBLEMAS_CONSEGUIR_CITA = "s19b"
PROBLEMAS_SER_ATENDIDO = "s19c"
PROBLEMAS_PAGAR_POR_COSTO = "s19d"
PROBLEMAS_ENTREGA_MEDICAMENTOS = "s19e"

casen_2022_grouped_servicio_df = (
    casen_2022_df.groupby(["capitulo", "nombre_capitulo"])
    .apply(calculate_servicio_metrics)
    .reset_index()
)

casen_2022_grouped_servicio_df

In [ ]:
print(casen_2022_grouped_servicio_df["total_fonasa"].sum())
print(casen_2022_grouped_servicio_df["problemas_llegar_consulta_si"].sum())

## Exportar CSV


In [ ]:
casen_2017_grouped_servicio_df["año"] = 2017
casen_2022_grouped_servicio_df["año"] = 2022

casen_2017_2022_grouped_servicio_df = pd.concat(
    [casen_2017_grouped_servicio_df, casen_2022_grouped_servicio_df],
    axis=0,
    ignore_index=True,
)

columnas_ordenadas = ["año"] + [
    col for col in casen_2017_2022_grouped_servicio_df.columns if col != "año"
]
casen_2017_2022_grouped_servicio_df = casen_2017_2022_grouped_servicio_df[
    columnas_ordenadas
]

casen_2017_2022_grouped_servicio_df.to_csv(
    "../../datasets/casen/casen_2017_2022_fonasa.csv", index=False
)